# 1. Recap Lecture 5: Random Forest.


![alt text](https://i.ibb.co/XJDGX3D/Screen-Shot-2020-10-26-at-17-31-17.png)

- Bagging + RSM allows to get more random trees, their error is more uncorrelated. That's why we can decrease the resulting error of the ensemble by averaging the predictions.
    - RMS: the best features is chosen from a different subset of the feature space for every tree.

- Error = Variance + Bias.
    - Variance: разброс.
    - Bias: смещенин
    
- Overfitting:
    - Small bias, since this tree "remmbers" quite well the seen data.
    - Big Variance.
    
- If the error are not strongly correlated, then the averaging of models' prediction allows us to decrease the variance of the resulting ensemble. При этом bias was already close to zero.


- Nice property of Random Forest: OOB.
    - We can estimate our model's quality in the unseen (excluded from the bootstrapped) data.
    - This will be an upper bound of the model's error.

![alt text](https://i.ibb.co/YDrYD00/Screen-Shot-2020-10-26-at-17-39-29.png)


---

# 2. Ensembles.

- An ensemble is contructed in the following way:
    - Take different models.
    - Train each of them independently, because we want the model to be independent from each other.
    - Predict with each of them.
    - Average the predictions and use these averaged predictions as output of the overall model.

That is, each model was trained independently from each other. But what if we want the models to help each other? This is the point where we introduce Boosting.

---

# 3. Boosting.

- Instead of training each model independently on the boostrapped data, we can train each model in sequence.
    - The goal is to decrease the model's resulting error.
    - The first model in some places made bad predictions.
    - The 2nd model says: "here and there you have the wrong predictions and should've been in this way".
    - And so on.

![alt text](https://i.ibb.co/jWyzyf1/Screen-Shot-2020-10-26-at-17-56-49.png)


- The first stump made some mistakes (blue points to the right).
- We can tell the next model (i.e. stump) to improve the mistakes of the previous one by increase their weight.
![alt text](https://i.ibb.co/HGcPkN1/Screen-Shot-2020-10-26-at-18-00-37.png)

- We constructed these models, trained them on the mistakes of the previous ones. Let's join them:

![alt text](https://i.ibb.co/hCSd1fM/Screen-Shot-2020-10-26-at-18-07-47.png)


- Let say for this problem we took $e^{-M}$ as loss function and try to find it for the $i$-th observation.
- Only the last model being trained at the moment is the one that can bring some improvement. The error made in the previous models is a constant, which only depends on the observation $i \implies w_i$.

![alt text](https://i.ibb.co/YZ4s3Vm/Screen-Shot-2020-10-26-at-18-24-33.png)

---

# 4. Gradient Boosting.

- Condition imposed on the lost function $L(y, f)$: it has to be differentiable.
- We consider$\mathbb{E}_{x,y}$ as the выворочная сдерняя.

![alt text](https://i.ibb.co/94VXNbF/Screen-Shot-2020-10-26-at-18-29-05.png)

- Suppose our optimal model is the sum of some optimal models from $0$ to $t-1$, where each model tries to improve the previous one. Let denote this model as the $(t-1)$-th ensemble.
- Now for the $t$-th ensemble we want to add the model $h(x, \theta)$, such that it minimize the error of the $(t-1)$-th ensemble. Then the t-th model is given by $\hat{f}_t(x)$:

![alt text](https://i.ibb.co/z5M3VBY/Screen-Shot-2020-10-26-at-18-40-24.png)

- We train the $t$-th model on the error of the $(t-1)$-th ensemble and the true target.
- Note: It's inappropiate to treat probability scores (like what we get in a classification task) as just numbers, since $p_{\text{true}} - p_{\text{predicted}} < 0$ and this can be taken as a probability error.

- $h(x, \theta)$ can be any model, even non-differentiable like Decision Tree. But here we talk about differentiability not wrt the model's parameters but wrt to the models themselves.
- **BUT** how we talk about differentiability wrt the models $f(x_i)$ when $L(y, f(x, \theta))$ depends on the parameters only, which in turn depends only on the known training data?
    - For this we calculate $r_{it}$ - the antigradient of the error function $L$ wrt to $f(x_i)$ (wrt the prediction).
    - We want the model $h(x_i, \theta)$ to approximate the best this antigradient $r_{it}$. For a fixed $x_i$ and $r_{it}$ we do that by finding $\theta_t$.

![alt text](https://i.ibb.co/HdQCSjL/Screen-Shot-2020-10-26-at-20-54-27.png)

- That is, we want to find model, such that the error is minimized. Mathematically we do this in this way:

![alt text](https://i.ibb.co/R48zv0S/Screen-Shot-2020-10-26-at-20-57-31.png)

![alt text](https://i.ibb.co/M8BJMcq/Screen-Shot-2020-10-26-at-23-23-31.png)

---

# 5. How to Construct Gradient Boostin Ensemble.

![alt text](https://i.ibb.co/jfk5SLQ/Screen-Shot-2020-10-26-at-23-27-09.png)

Example:

 ![alt text](https://i.ibb.co/C8zrBfV/Screen-Shot-2020-10-26-at-23-25-41.png)
 ![alt text](https://i.ibb.co/bzvmKx1/Screen-Shot-2020-10-26-at-23-30-03.png)
 
 ---
 
 # 6. Why is not a good idea to use deep trees with gradient boosting.
 
- If we have a deep tree, then probably is overfitting, which implies that its error is already quite small. If the error is small $\implies$ the gradient is also quite small $\implies$ doing the gradient boosting step makes no sense afterwards.
- In general, if at some step $t$ we overfitted, there is no need to continue with gradient boosting iterations, since there is no error to train on.
- With random forest we can use deep trees, since in this case fitting is done independently (there is no antigradient of error function to calculate from).

---

## 6.1. Вопрос в собеседовании.

- Насколько осмысленно построение ансамбля типа бустинг в задаче регресси если каждые предиктор/модель $h(x, \theta) = \theta^{\top}x$ - это линейная модель.
    - Ответ: не очень. Потому что $\hat{f} = \sum\limits_{t=0}^T \rho_t h_t = \sum\limits_{t=0}^T \rho_t \theta^{\top}_tx = \left(\sum\limits_{t=0}^T \rho_t \theta^{\top}_t\right)x = \tilde{\theta}^{\top}x$. That is the resulting boosting $\hat{f}$ is just a linear model also (linear combination of linear models is also a linear model). By doing boosting in this case we don't leave the class of linear models. We only complicate the search of the solution.
- Почему логистическая регрессия может решить задачу классификации для линейной **не**разделимой если лог.рег. описывает линейную разделяющую гиперповерхность? 
    - Ответ: Это потому что лог.рег. представляет собой нелинейное отображение, т.е $\sigma(\theta^{\top}x)$. So when we construct a boosting-like ensemble, we construct a linear combination of nonlinear models, that why the result can be nonlinear.


---

# 7. Gradient Boosting: Technical Issues.

![alt text](https://i.ibb.co/tzHgNqB/Screen-Shot-2020-10-27-at-00-29-51.png)

---

# 8. Blending.

![alt text](https://i.ibb.co/6J3Z7Dw/Screen-Shot-2020-10-27-at-00-40-12.png)

- Instead of just (macro) averaging the base models' predictions we use a weighted averaging, taking into account how well the base model made predictions.
- The question now is how to find those weights.
- $\implies$ we want to construct a linear model on the base models' predictions. How we do this.

- Let's divide the dataset into two parts:
    - Part A: we train our base models.
    - Part B: we try to find these weights by
        - Forming $\hat{f}(x) = \sum\limits_{i=1}^M \rho_if_i(x)$.
        - Fitting $\hat{f}(x)$ on B.

![alt text](https://i.ibb.co/Q8VC6MV/Screen-Shot-2020-10-27-at-00-51-57.png)

![alt text](https://i.ibb.co/sm0JnKK/Screen-Shot-2020-10-27-at-00-53-30.png)

---

# 9. Stacking.

- It fixes the problem of having to split the data with blending.

![alt text](https://i.ibb.co/THysjM4/Screen-Shot-2020-10-27-at-00-59-12.png)

- Train base algorithm(s) on different groups of folds leaving one fold out.
- Predict the meta-features on the left-out fold and test data.
- Train the meta-algorithm on the meta-features representation of the train data.
- Use it on the meta-features representation of the test data

---

## 9.1. Stacking: Pros and Cons.

- Pros:
    - Powerful ensembling method, if you know how to use it.
    - Quite popular in ML-competitions.
    - One might perform stacking on the meta-features dataset as well.


- Cons:
    - Meta-features on each fold are actually predicted by different models.
    - However, regularization usually helps.
    - Hard to explain your model behaviour.
 